# Exercise 04 : Enrichment and transformations

## Imports

In [1]:
import pandas as pd
import numpy as np
import requests

## Read the JSON file that you saved in ex02

In [2]:
df = pd.read_json('../data/auto.json', orient='records')
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.0,Ford,Focus
1,E432XX77RUS,1,6500.0,Toyota,Camry
2,7184TT36RUS,1,2100.0,Ford,Focus
3,X582HE161RUS,2,2000.0,Ford,Focus
4,92918M178RUS,1,5700.0,Ford,Focus


## One of the columns has the float type, so let us define the format of it in pandas using pd.options.display.float_format: floats should be displayed with two decimals

In [3]:
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
720,Y163O8161RUS,2,1600.00,Ford,Focus
721,M0309X197RUS,1,22300.00,Ford,Focus
722,O673E8197RUS,2,600.00,Ford,Focus
723,8610T8154RUS,1,2000.00,Ford,Focus


## Enrich the dataframe using a sample from that dataframe
- create a sample with 200 new observations with random_state = 21

In [5]:
sample = df.sample(200, random_state=21)
sample.head()

,CarNumber,Refund,Fines,Make,Model
445,M0299X197RUS,2,19200.00,Ford,Focus
22,83298C154RUS,2,8594.59,Ford,Focus
93,H957HY161RUS,1,2000.00,Ford,Focus
173,T941CC96RUS,1,2000.00,Ford,Focus
697,H966HY161RUS,1,500.00,Ford,Focus


- the sample should not have new combinations of the car number, make and model, so the whole dataset will be consistent in these terms
- there are no restrictions on the refund and fines, you can take any value from these columns at random and use it towards any car number

In [6]:
sample['Refund'] = np.random.choice(df['Refund'], size=len(sample))
sample['Fines'] = np.random.choice(df['Fines'], size=len(sample))

In [7]:
sample

,CarNumber,Refund,Fines,Make,Model
445,M0299X197RUS,2,900.00,Ford,Focus
22,83298C154RUS,2,15300.00,Ford,Focus
93,H957HY161RUS,1,8594.59,Ford,Focus
173,T941CC96RUS,1,2500.00,Ford,Focus
697,H966HY161RUS,2,8500.00,Ford,Focus
...,...,...,...,...,...
14,8182XX154RUS,2,2200.00,Ford,Focus
623,X796TH96RUS,1,9500.00,Ford,Focus
498,T011MY163RUS,2,8594.59,Ford,Focus
536,T341CC96RUS,1,500.00,Volkswagen,Passat


## Concatenate the sample with the initial dataframe to a new dataframe concat_rows

In [8]:
concat_rows = pd.concat([df, sample], ignore_index=True)

In [9]:
concat_rows

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
920,8182XX154RUS,2,2200.00,Ford,Focus
921,X796TH96RUS,1,9500.00,Ford,Focus
922,T011MY163RUS,2,8594.59,Ford,Focus
923,T341CC96RUS,1,500.00,Volkswagen,Passat


In [10]:
concat_rows.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        914
dtype: int64

## Enrich the dataframe concat_rows by a new column with the data generated
- create a series with the name Year using random integers from 1980 to 2019
- use np.random.seed(21) before generating the years

In [11]:
np.random.seed(21)
year = pd.Series(np.random.randint(1980, 2019, size=len(concat_rows)), name='Year')
year

0      1989
1      1995
2      1984
3      2015
4      2014
       ... 
920    1996
921    2002
922    1996
923    2012
924    1984
Name: Year, Length: 925, dtype: int64

## Concatenate the series with the dataframe and name it fines

In [12]:
fines = pd.concat([concat_rows, year], axis='columns')
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
920,8182XX154RUS,2,2200.00,Ford,Focus,1996
921,X796TH96RUS,1,9500.00,Ford,Focus,2002
922,T011MY163RUS,2,8594.59,Ford,Focus,1996
923,T341CC96RUS,1,500.00,Volkswagen,Passat,2012


In [13]:
fines.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        914
Year         925
dtype: int64

## Enrich the dataframe with the data from another dataframe
- get the most popular surnames(you can find the file surname.json in the attachments) in the US

In [14]:
surnames = pd.read_json('../data/surname.json', orient='values')
surnames.columns = surnames.iloc[0]
surnames.drop(surnames.index[0], inplace=True)
surnames.head()

,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
5,BAILEY,277845,72


## Create a new series with the surnames (they should not have special characters like commas, brackets, etc.) from the data you gathered, the count should be equal to the number of unique car numbers using the sample (use random_state = 21)

In [15]:
surnames = surnames.loc[surnames['NAME'].str.isalpha() == True]
surnames_sample = surnames.sample(fines['CarNumber'].unique().shape[0], random_state=21, replace=True)['NAME']
surnames_sample

74    RICHARDSON
80          ROSS
57        MORGAN
5         BAILEY
49         LOPEZ
         ...    
10      CAMPBELL
32          HALL
6          BAKER
21          DIAZ
57        MORGAN
Name: NAME, Length: 531, dtype: object

In [16]:
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
920,8182XX154RUS,2,2200.00,Ford,Focus,1996
921,X796TH96RUS,1,9500.00,Ford,Focus,2002
922,T011MY163RUS,2,8594.59,Ford,Focus,1996
923,T341CC96RUS,1,500.00,Volkswagen,Passat,2012


## Create the dataframe owners with 2 columns: CarNumber and SURNAME

In [17]:
owners = pd.DataFrame(zip(fines['CarNumber'].unique(), surnames_sample), columns=['CarNumber', 'SURNAME'])
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


## Append 5 more observations to the fines dataframe (come up with your own ideas of CarNumber, etc.)

In [18]:
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
920,8182XX154RUS,2,2200.00,Ford,Focus,1996
921,X796TH96RUS,1,9500.00,Ford,Focus,2002
922,T011MY163RUS,2,8594.59,Ford,Focus,1996
923,T341CC96RUS,1,500.00,Volkswagen,Passat,2012


In [19]:
my_data = pd.DataFrame({'CarNumber': ['K089PY178RUS', 'C718MC178RUS', 'K361KA178RUS', 
                                 'O432AB178RUS', 'X023HA178RUS'],
                   'Refund': [1, 2, 3, 4, 5],
                   'Fines': [1234.00, 4321.00, 2345.00, 5432.00, 2121.00],
                   'Make': ['Ford', 'Ford', 'Ford', 'Ford', 'Ford'],
                   'Model': ['Mustang', 'Mustang', 'Mustang', 'Mustang', 'Mustang',],
                   'Year': [1969, 1969, 1969, 1969, 1969]})

fines = pd.concat([fines, my_data], ignore_index=True)
fines


,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
925,K089PY178RUS,1,1234.00,Ford,Mustang,1969
926,C718MC178RUS,2,4321.00,Ford,Mustang,1969
927,K361KA178RUS,3,2345.00,Ford,Mustang,1969
928,O432AB178RUS,4,5432.00,Ford,Mustang,1969


## Delete the dataframe last 20 observations from the owners and add 3 new observations (they are not the same as those you add to the fines dataframe)

In [20]:
owners.drop(owners.tail(20).index, inplace=True)
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
506,T914CT197RUS,HERNANDEZ
507,E41977152RUS,BAKER
508,9464EX178RUS,MARTIN
509,O50197197RUS,WRIGHT


In [21]:
my_data = pd.DataFrame({'CarNumber': ['E778MX05RUS', 'H973EE05RUS', 'M558MB05RUS'],
                       'SURNAME': ['KADYROV', 'KADYROV', 'KADYROV']})
owners = pd.concat([owners, my_data], ignore_index=True)
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
509,O50197197RUS,WRIGHT
510,7608EE777RUS,HILL
511,E778MX05RUS,KADYROV
512,H973EE05RUS,KADYROV


## Join both dataframes
- the new dataframe should have only the car numbers that exist in both dataframes

In [22]:
pd.merge(fines, owners, how='inner', left_on='CarNumber', right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,Y163O8161RUS,2,1600.00,Ford,Focus,1999,RICHARDSON
2,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
3,E432XX77RUS,2,13000.00,Toyota,Camry,1992,ROSS
4,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
...,...,...,...,...,...,...,...
894,E41977152RUS,2,2400.00,Ford,Focus,2001,BAKER
895,9464EX178RUS,2,2100.00,Ford,Focus,1993,MARTIN
896,O50197197RUS,2,7800.00,Ford,Focus,1986,WRIGHT
897,7608EE777RUS,1,4000.00,Skoda,Octavia,2013,HILL


 - the new dataframe should have all the car numbers that exist in both dataframes

In [23]:
pd.merge(fines, owners, how='outer', left_on='CarNumber', right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1999.00,RICHARDSON
2,E432XX77RUS,1.00,6500.00,Toyota,Camry,1995.00,ROSS
3,E432XX77RUS,2.00,13000.00,Toyota,Camry,1992.00,ROSS
4,7184TT36RUS,1.00,2100.00,Ford,Focus,1984.00,MORGAN
...,...,...,...,...,...,...,...
928,O432AB178RUS,4.00,5432.00,Ford,Mustang,1969.00,NaN
929,X023HA178RUS,5.00,2121.00,Ford,Mustang,1969.00,NaN
930,E778MX05RUS,NaN,NaN,NaN,NaN,NaN,KADYROV
931,H973EE05RUS,NaN,NaN,NaN,NaN,NaN,KADYROV


 - the new dataframe should have only the car numbers from the fines dataframe

In [24]:
pd.merge(fines, owners, how='left', left_on='CarNumber', right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
925,K089PY178RUS,1,1234.00,Ford,Mustang,1969,NaN
926,C718MC178RUS,2,4321.00,Ford,Mustang,1969,NaN
927,K361KA178RUS,3,2345.00,Ford,Mustang,1969,NaN
928,O432AB178RUS,4,5432.00,Ford,Mustang,1969,NaN


- the new dataframe should have only the car numbers from the owners dataframe

In [25]:
pd.merge(fines, owners, how='right', left_on='CarNumber', right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1999.00,RICHARDSON
2,E432XX77RUS,1.00,6500.00,Toyota,Camry,1995.00,ROSS
3,E432XX77RUS,2.00,13000.00,Toyota,Camry,1992.00,ROSS
4,7184TT36RUS,1.00,2100.00,Ford,Focus,1984.00,MORGAN
...,...,...,...,...,...,...,...
897,7608EE777RUS,1.00,4000.00,Skoda,Octavia,2013.00,HILL
898,7608EE777RUS,2.00,1100.00,Skoda,Octavia,1987.00,HILL
899,E778MX05RUS,NaN,NaN,NaN,NaN,NaN,KADYROV
900,H973EE05RUS,NaN,NaN,NaN,NaN,NaN,KADYROV


## Create a pivot table from the fines dataframe, it should look like this (the values are the sums of the fines), but with all the years (the values may be different for you):

In [26]:
pd.pivot_table(fines,
               index=['Make', 'Model'],
               values='Fines',
               columns='Year')

Year                  1969     1980     1981     1982     1983    1984  \
Make       Model                                                         
Ford       Focus       NaN  5066.67 15130.72  5199.73 10830.35 9324.66   
           Mondeo      NaN      NaN      NaN 46200.00      NaN     NaN   
           Mustang 3090.60      NaN      NaN      NaN      NaN     NaN   
Skoda      Octavia     NaN 15097.29  1900.00  4447.29      NaN 4447.29   
Toyota     Camry       NaN 12000.00      NaN  8594.59  1000.00 1000.00   
           Corolla     NaN      NaN  6800.00      NaN  6400.00     NaN   
Volkswagen Golf        NaN 20800.00  8594.59  5000.00   200.00     NaN   
           Jetta       NaN      NaN  1000.00      NaN      NaN     NaN   
           Passat      NaN   900.00  1750.00      NaN   550.00 8594.59   
           Touareg     NaN      NaN      NaN      NaN      NaN     NaN   

Year                    1985     1986     1987     1988  ...    2009     2010  \
Make       Model                                         ...                    
Ford       Focus    10978.72  6184.62  4204.98  4005.60  ... 5709.27  5658.82   
           Mondeo        NaN      NaN      NaN      NaN  ...     NaN      NaN   
           Mustang       NaN      NaN      NaN      NaN  ...     NaN      NaN   
Skoda      Octavia   7297.29      NaN  1550.00  1700.00  ...     NaN  2833.33   
Toyota     Camry         NaN 19800.00      NaN      NaN  ...     NaN 22400.00   
           Corolla    500.00      NaN 27150.00      NaN  ... 8594.59  3000.00   
Volkswagen Golf    168000.00      NaN  2400.00      NaN  ...     NaN      NaN   
           Jetta     9000.00      NaN      NaN 46000.00  ...     NaN      NaN   
           Passat        NaN  8000.00  2000.00  8594.59  ...  800.00  9500.00   
           Touareg       NaN      NaN      NaN      NaN  ... 5800.00      NaN   

Year                  2011    2012     2013     2014    2015     2016    2017  \
Make       Model                                                                
Ford       Focus   5804.73 6949.46 11806.34  5321.32 8195.24  7255.57 5176.78   
           Mondeo      NaN     NaN 20550.00      NaN     NaN      NaN 8600.00   
           Mustang     NaN     NaN      NaN      NaN     NaN      NaN     NaN   
Skoda      Octavia 3000.00  850.00  3933.33   800.00 5464.86 11900.00 2400.00   
Toyota     Camry       NaN 7500.00      NaN      NaN     NaN  7800.00     NaN   
           Corolla  900.00     NaN      NaN      NaN 1000.00      NaN 2400.00   
Volkswagen Golf        NaN     NaN      NaN  6950.00 5200.00      NaN     NaN   
           Jetta       NaN     NaN      NaN      NaN     NaN      NaN     NaN   
           Passat      NaN  500.00  1600.00 18300.00     NaN      NaN     NaN   
           Touareg     NaN     NaN      NaN      NaN     NaN      NaN     NaN   

Year                   2018  
Make       Model             
Ford       Focus    6488.89  
           Mondeo       NaN  
           Mustang      NaN  
Skoda      Octavia 76600.00  
Toyota     Camry        NaN  
           Corolla      NaN  
Volkswagen Golf     9300.00  
           Jetta        NaN  
           Passat       NaN  
           Touareg      NaN  

[10 rows x 40 columns]

## Save both the fines and owners dataframes to CSV files without an index

In [27]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)